In [ ]:
import requests
import zipfile
import pandas as pd
zip_local_path = r'/tmp/ml-latest-small.zip'

###Download zip file

In [ ]:
zip_url = r'https://u.netology.ngcdn.ru/backend/uploads/lms/attachments/files/data/15257/ml-latest-small.zip'
zip_data = requests.get(zip_url)
with open(zip_local_path, 'w+b') as zip_file:
    zip_file.write(zip_data.content)
    zip_file.flush()

# Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:

- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг
*italicized text*
Результат классификации запишите в столбец class

#### get `ratings_df`, `movies_df`

In [ ]:
with zipfile.ZipFile(zip_local_path, mode='r', compression=zipfile.ZIP_DEFLATED) as archive:
    with archive.open(r'ml-latest-small/ratings.csv', mode='r') as ratings_stream:
        ratings_df = pd.read_csv(filepath_or_buffer=ratings_stream, header=0)
    with archive.open(r'ml-latest-small/movies.csv', mode='r') as movies_stream:
        movies_df = pd.read_csv(filepath_or_buffer=movies_stream, header=0)

In [ ]:
ratings_df.head(9)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125


In [ ]:
movies_df.head(9)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action


#### solution

In [ ]:
movie_ratings_df = movies_df.merge(ratings_df, how='left', left_on=['movieId'], right_on=['movieId'], validate='1:m')

In [ ]:
movie_ratings_df.head(9)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,3.0,8.518667e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9.0,4.0,9.386292e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13.0,5.0,1.331380e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.0,9.979383e+08
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19.0,3.0,8.551901e+08
5,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,20.0,3.5,1.238730e+09
6,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,23.0,3.0,1.148730e+09
7,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,26.0,5.0,1.360088e+09
8,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,30.0,4.0,9.449431e+08


In [ ]:
def classify_movie(rating_col):
    if (rating_col <= 2.0):
        return 'низкий рейтинг'
    if (rating_col <= 4.0):
        return 'средний рейтинг'
    return 'высокий рейтинг'

In [ ]:
movies_classified = movie_ratings_df.groupby(['title']).agg({'rating': 'mean'})
movies_classified['class'] = movies_classified['rating'].apply(classify_movie)

In [ ]:
movies_classified.head(9)

,rating,class
title,,
"""Great Performances"" Cats (1998)",1.750000,низкий рейтинг
$9.99 (2008),3.833333,средний рейтинг
'Hellboy': The Seeds of Creation (2004),2.000000,низкий рейтинг
'Neath the Arizona Skies (1934),0.500000,низкий рейтинг
'Round Midnight (1986),2.250000,средний рейтинг
'Salem's Lot (2004),3.500000,средний рейтинг
'Til There Was You (1997),2.625000,средний рейтинг
"'burbs, The (1989)",3.052632,средний рейтинг
'night Mother (1986),5.000000,высокий рейтинг


In [ ]:
movies_classified.value_counts()

rating    class          
4.000000  средний рейтинг    961
3.000000  средний рейтинг    848
3.500000  средний рейтинг    776
2.500000  средний рейтинг    444
2.000000  низкий рейтинг     377
                            ... 
3.704545  средний рейтинг      1
3.704082  средний рейтинг      1
3.703947  средний рейтинг      1
3.702381  средний рейтинг      1
3.527778  средний рейтинг      1
Length: 1280, dtype: int64

# Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
```
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
```
Результат классификации запишите в отдельный столбец region.

#### get `keywords_df`

In [ ]:
with zipfile.ZipFile(zip_local_path, mode='r', compression=zipfile.ZIP_DEFLATED) as archive:
    with archive.open(r'ml-latest-small/keywords.csv', mode='r') as keywords_stream:
        keywords_df = pd.read_csv(filepath_or_buffer=keywords_stream, header=0)

In [ ]:
keywords_df.head(9)

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195
5,одноклассники моя страница,19976138
6,майл,15144673
7,авито,14786593
8,переводчик,14453469


#### solution

In [ ]:
def assign_region(row):
    geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск'],
    }
    keywords = row['keyword'].lower()
    for region, cities in geo_data.items():
        for city in cities:
            if (city in keywords):
                return region
    return 'undefined'

In [ ]:
keywords_df['region'] = keywords_df.apply(assign_region, axis=1)

In [ ]:
keywords_df.groupby(['region']).count()

,keyword,shows
region,,
undefined,99253,99253
Дальний Восток,94,94
Северо-Запад,266,266
Центр,387,387


# Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы grouplens.org.../movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    1. для каждой строки пройдите по всем годам списка years
        1. если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
        2. если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга


#### get `ratings_df`, `movies_df`

In [ ]:
with zipfile.ZipFile(zip_local_path, mode='r', compression=zipfile.ZIP_DEFLATED) as archive:
    with archive.open(r'ml-latest-small/ratings.csv', mode='r') as ratings_stream:
        ratings_df = pd.read_csv(filepath_or_buffer=ratings_stream, header=0)
    with archive.open(r'ml-latest-small/movies.csv', mode='r') as movies_stream:
        movies_df = pd.read_csv(filepath_or_buffer=movies_stream, header=0)

In [ ]:
ratings_df.head(9)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125


In [ ]:
movies_df.head(9)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action


#### solution

In [ ]:
years = dict(zip(range(1950, 2011), map(str, range(1950, 2011))))
f'years min = {min(years.keys())}, max = {max(years.keys())}, first = {list(years.items())[0]}'

"years min = 1950, max = 2010, first = (1950, '1950')"

In [ ]:
def get_production_year(title_col):
    for year_num, year_str in years.items():
        if (year_str in title_col):
            return year_num
    return 1900

In [ ]:
movies_df['year'] = movies_df['title'].apply(get_production_year)

In [ ]:
movies_df.head(9)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995
5,6,Heat (1995),Action|Crime|Thriller,1995
6,7,Sabrina (1995),Comedy|Romance,1995
7,8,Tom and Huck (1995),Adventure|Children,1995
8,9,Sudden Death (1995),Action,1995


In [ ]:
year_rating_df = movies_df.merge(ratings_df, how = 'left', left_on = ['movieId'], right_on = ['movieId'], validate='1:m').groupby(['year']).agg({'rating': 'mean'}).sort_values(['rating'], ascending = False)

In [ ]:
year_rating_df

,rating
year,
1957,4.014241
1972,4.011136
1952,4.000000
1954,3.994220
1951,3.983539
...,...
2005,3.448434
2003,3.444777
1996,3.426600
